In [15]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Minimum degree (defines the range for number of keys)
        self.leaf = leaf
        self.keys = []
        self.children = []

    def traverse(self):
        for i in range(len(self.keys)):
            if not self.leaf:
                self.children[i].traverse()
            print(self.keys[i], end=" ")
        if not self.leaf:
            self.children[-1].traverse()

    def search(self, k):
        i = 0
        while i < len(self.keys) and k > self.keys[i]:
            i += 1
        if i < len(self.keys) and self.keys[i] == k:
            return self
        if self.leaf:
            return None
        return self.children[i].search(k)

    def insert_non_full(self, k):
        i = len(self.keys) - 1
        if self.leaf:
            self.keys.append(0)
            while i >= 0 and k < self.keys[i]:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = k
        else:
            while i >= 0 and k < self.keys[i]:
                i -= 1
            i += 1
            if len(self.children[i].keys) == 2 * self.t - 1:
                self.split_child(i, self.children[i])
                if k > self.keys[i]:
                    i += 1
            self.children[i].insert_non_full(k)

    def split_child(self, i, y):
        t = self.t
        z = BTreeNode(t, y.leaf)
        z.keys = y.keys[t:]  # Last t-1 keys go to new node
        y.keys = y.keys[:t - 1]  # First t-1 keys remain
        if not y.leaf:
            z.children = y.children[t:]
            y.children = y.children[:t]
        self.children.insert(i + 1, z)
        self.keys.insert(i, y.keys.pop())


In [18]:
class BTree:
    def __init__(self, t):
        self.t = t
        self.root = BTreeNode(t, leaf=True)

    def traverse(self):
        if self.root:
            self.root.traverse()
            print()

    def search(self, k):
        return self.root.search(k)

    def insert(self, k):
        root = self.root
        if len(root.keys) == 2 * self.t - 1:
            temp = BTreeNode(self.t, leaf=False)
            temp.children.insert(0, root)
            temp.split_child(0, root)
            i = 0
            if temp.keys[0] < k:
                i += 1
            temp.children[i].insert_non_full(k)
            self.root = temp
        else:
            root.insert_non_full(k)


In [20]:
if __name__ == '__main__':
    print("B-tree with t=3 (min degree)")
    btree = BTree(t=3)

    values = [10, 20, 5, 6, 12, 30, 7, 17]
    for val in values:
        btree.insert(val)

    print("Traversal of the B-tree:")
    btree.traverse()

    key = 6
    result = btree.search(key)
    print(f"\nSearch for {key}:", "Found" if result else "Not found")


B-tree with t=3 (min degree)
Traversal of the B-tree:
5 6 7 12 17 20 30 

Search for 6: Found
